In [1]:
# Import the packages needed
import numpy as np, pandas as pd
import urllib
import json

In [2]:
# Load the data as a JSON dict using the API

line_to_scrape = '14'

with urllib.request.urlopen("http://api.511.org/transit/timetable?api_key=f5c9c70c-3e13-4e76-8481-d3ecc866ef3f&operator_id=SF&line_id="+line_to_scrape+"&format=json") as url:
    data = json.load(url)

In [3]:
# Example of the data architecture:
# This is a single stop ID (Mission St & 11th St)
# View the HTML time table at https://511.org/transit/schedules-agency-info/agency/SF/schedules/route/14 to compare
# Stop IDs are located at:
int(data['Content']['TimetableFrame'][1]['vehicleJourneys']['ServiceJourney'][0]['calls']['Call'][0]['ScheduledStopPointRef'].get('ref'))

15544

In [4]:
# This is one set of bus trips for a given service journey, in this case item 1 of the first time table
data['Content']['TimetableFrame'][0]['vehicleJourneys']['ServiceJourney'][0]['calls']

{'Call': [{'order': '1',
   'ScheduledStopPointRef': {'ref': '15544'},
   'Arrival': {'Time': '05:09:00', 'DaysOffset': '0'},
   'Departure': {'Time': '05:09:00', 'DaysOffset': '0'}},
  {'order': '2',
   'ScheduledStopPointRef': {'ref': '15535'},
   'Arrival': {'Time': '05:14:00', 'DaysOffset': '0'},
   'Departure': {'Time': '05:14:00', 'DaysOffset': '0'}},
  {'order': '3',
   'ScheduledStopPointRef': {'ref': '16498'},
   'Arrival': {'Time': '05:21:00', 'DaysOffset': '0'},
   'Departure': {'Time': '05:21:00', 'DaysOffset': '0'}}]}

In [5]:
# Loops through each time table and for each trip id, prints the stop id and arrival time

# First I will make a dataframe with all of the time tables and then think about splitting them by time table or just reassign the vars for that later

lines, directions, timetables, trip_ids, stop_ids, arrival_times = [], [], [], [], [], []

for timetable in data['Content']['TimetableFrame']:
    for servicejourney in timetable['vehicleJourneys']['ServiceJourney']:
        for call in servicejourney['calls']['Call']:

            # Grab the line
            line = timetable['Name'].split(':')
            lines.append(int(line[0]))
            
            # Inbound or outbound
            directions.append(line[1])
            
            # What time table it is
            timetables.append(line[2])
            
            # What the trip id is
            trip = servicejourney['id']
            trip_ids.append(trip)
            
            # What the stop id is
            stop = call['ScheduledStopPointRef']['ref']
            stop_ids.append(stop)
            
            # Grabs the arrival time
            arrival = call['Arrival']['Time']
            arrival_times.append(arrival)

In [6]:
tidy_df = pd.DataFrame({'line':lines,
                       'direction':directions,
                       'timetable':timetables,
                       'trip':trip_ids,
                       'stop_id':stop_ids,
                       'arrival_time':arrival_times})

tidy_df.head()

,line,direction,timetable,trip,stop_id,arrival_time
0,14,IB,Saturday,8529268,15544,05:09:00
1,14,IB,Saturday,8529268,15535,05:14:00
2,14,IB,Saturday,8529268,16498,05:21:00
3,14,IB,Saturday,8529267,15544,05:31:00
4,14,IB,Saturday,8529267,15535,05:37:00


In [7]:
lines = pd.Series(lines)
directions = pd.Series(directions)
timetables = pd.Series(timetables)
trips = pd.Series(trip_ids)
stops = pd.Series(stop_ids)
arrivals = pd.Series(arrival_times)

arrivals = pd.to_datetime(arrivals)

df = pd.DataFrame({'lines' : lines,
                   'directions' : directions,
                   'timetables' : timetables,
                   'trips' : trips,
                   'stops' : stops,
                   'TIME' : arrivals})

In [8]:
df.head()

,lines,directions,timetables,trips,stops,TIME
0,14,IB,Saturday,8529268,15544,2019-03-17 05:09:00
1,14,IB,Saturday,8529268,15535,2019-03-17 05:14:00
2,14,IB,Saturday,8529268,16498,2019-03-17 05:21:00
3,14,IB,Saturday,8529267,15544,2019-03-17 05:31:00
4,14,IB,Saturday,8529267,15535,2019-03-17 05:37:00


In [9]:
import os

filePath = os.path.join(r'../data','route_' + line_to_scrape + '_timetables_data' + '.csv')
os.path.isfile(filePath)
df.to_csv(filePath, index=False)